In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

scanners = {
    "EpsonV500": "/content/drive/MyDrive/TraceFinder_Project/Train_Data/EpsonV500",
    "Canon200": "/content/drive/MyDrive/TraceFinder_Project/Train_Data/Canon220"
}

for name, path in scanners.items():
    output_folder = f"/content/dataset/{name}"
    os.makedirs(output_folder, exist_ok=True)

    if os.path.exists(path):
        zip_files = sorted([os.path.join(path, f) for f in os.listdir(path) if f.endswith('.zip')])

        print(f"Starting {name}...")
        for i, z_path in enumerate(zip_files):
            with zipfile.ZipFile(z_path, 'r') as z_ref:
                inner_files = [f for f in z_ref.namelist() if not f.endswith('/')]
                for f_name in inner_files:
                    clean_name = os.path.basename(f_name)
                    unique_name = f"set{i+1}_{clean_name}"

                    with z_ref.open(f_name) as source, open(os.path.join(output_folder, unique_name), "wb") as target:
                        target.write(source.read())

        print(f"{name} finished. Count: {len(os.listdir(output_folder))}")
    else:
        print(f"Path not found for {name}: {path}")

print("\nALL SCANNERS PROCESSED SUCCESSFULLY!")

Starting EpsonV500...
EpsonV500 finished. Count: 208
Starting Canon200...
Canon200 finished. Count: 516

ALL SCANNERS PROCESSED SUCCESSFULLY!


In [ ]:
import cv2
import os
import numpy as np

input_root = '/content/dataset/'
output_root = '/content/preprocessed_dataset/'
target_size = (224, 224)
if not os.path.exists(output_root):
    os.makedirs(output_root)


for scanner_folder in os.listdir(input_root):
    scanner_path = os.path.join(input_root, scanner_folder)

    if os.path.isdir(scanner_path):
        output_scanner_path = os.path.join(output_root, scanner_folder)
        os.makedirs(output_scanner_path, exist_ok=True)

        print(f"Processing: {scanner_folder}")

        for filename in os.listdir(scanner_path):
            if filename.endswith(('.tif', '.jpg', '.png')):
                img_path = os.path.join(scanner_path, filename)
                img = cv2.imread(img_path)

                if img is not None:

                    img_resized = cv2.resize(img, target_size)


                    img_normalized = img_resized / 255.0


                    save_path = os.path.join(output_scanner_path, filename)
                    cv2.imwrite(save_path, (img_normalized * 255).astype(np.uint8))

print("Preprocessing complete!")

Processing: EpsonV500
Processing: Canon200
Preprocessing complete!


In [ ]:
import shutil
import os

source_folder = '/content/preprocessed_dataset'


destination_folder = '/content/drive/MyDrive/TraceFinder_Project/Final_Preprocessed_Data'


if os.path.exists(source_folder):
    shutil.copytree(source_folder, destination_folder, dirs_exist_ok=True)
    print("done")
else:
    print("!done.")

done


In [ ]:
import os
import shutil

source_folder = '/content/preprocessed_dataset'
destination_folder = '/content/drive/MyDrive/TraceFinder_Project/Final_Dataset'

if os.path.exists(source_folder):
    for scanner in os.listdir(source_folder):
        src_scanner_path = os.path.join(source_folder, scanner)
        dst_scanner_path = os.path.join(destination_folder, scanner)

        if not os.path.exists(dst_scanner_path):
            os.makedirs(dst_scanner_path)
            print(f"Created folder: {scanner}")

        files = os.listdir(src_scanner_path)
        print(f"Copying {len(files)} files for {scanner}...")

        for file_name in files:
            src_file = os.path.join(src_scanner_path, file_name)
            dst_file = os.path.join(dst_scanner_path, file_name)

            try:
                shutil.copy2(src_file, dst_file)
            except Exception as e:
                print(f"Could not copy {file_name}: {e}")

    print("\nAll folders processed and backed up to Drive!")
else:
    print("Source folder not found.")

Created folder: EpsonV500
Copying 208 files for EpsonV500...
Created folder: Canon200
Copying 516 files for Canon200...

All folders processed and backed up to Drive!
